In [ ]:
defaulter_schema="member_id string,delinq_2yrs float,delinq_amnt float,pub_rec float,pub_rec_bankruptcies float,inq_last_6mths float,total_rec_late_fee float,mths_since_last_delinq float,mths_since_last_record float"

In [ ]:
srv_date = dt.datetime.now().strftime('%Y-%m-%d')
print(srv_date)
bv_lndf_Path = f"/user/{user_name}/lendingClub/srv/2024-01-07/loan_defaulters/"
print(bv_lndf_Path)
delinq_csv_dest = f"/user/{user_name}/lendingClub/bv/csv/loan_defaulters_delinq/"
delinq_parq_dest = f"/user/{user_name}/lendingClub/bv/parquet/loan_defaulters_delinq/"

rec_csv_dest = f"/user/{user_name}/lendingClub/bv/csv/loan_defaulters_rec/"
rec_parq_dest = f"/user/{user_name}/lendingClub/bv/parquet/loan_defaulters_rec/"

rec_enq_csv_dest = f"/user/{user_name}/lendingClub/bv/csv/loan_defaulters_rec_enq/"
rec_enq_parq_dest = f"/user/{user_name}/lendingClub/bv/parquet/loan_defaulters_rec_enq/"

print(delinq_csv_dest)
print(rec_parq_dest)

In [ ]:
lndf_DF = spark.read \
            .format("csv") \
            .option("header", True) \
            .schema(defaulter_schema) \
            .option("path", bv_lndf_Path) \
            .load()

dl2y_DF = lndf_DF.withColumn("delinq_2yrs",col("delinq_2yrs").cast("integer")).fillna(0,subset=["delinq_2yrs"])
pub_rec_DF = dl2y_DF.withColumn("pub_rec",col("pub_rec").cast("integer")).fillna(0,subset=["pub_rec"])
bnkrp_DF = pub_rec_DF.withColumn("pub_rec_bankruptcies",col("pub_rec_bankruptcies").cast("integer")).fillna(0,subset=["pub_rec_bankruptcies"])
mths_DF = bnkrp_DF.withColumn("inq_last_6mths",col("inq_last_6mths").cast("integer")).fillna(0,subset=["inq_last_6mths"])


delinq_DF = mths_DF.select("member_id", "delinq_2yrs", "delinq_amnt", col("mths_since_last_delinq").cast("int")) \
                    .filter((col("delinq_2yrs") > 0) | (col("mths_since_last_delinq")>0 ) )

In [ ]:
pub_rec = mths_DF.select("member_id") \
                .filter( (col("pub_rec")> 0.0) | \
                         (col("pub_rec_bankruptcies") >0.0) | \
                         (col("inq_last_6mths") > 0.0)
                       )

In [ ]:
pub_rec_enq_det = mths_DF.select("member_id","pub_rec","pub_rec_bankruptcies","inq_last_6mths") \
                .filter( (col("pub_rec")> 0.0) | \
                         (col("pub_rec_bankruptcies") >0.0) | \
                         (col("inq_last_6mths") > 0.0)
                       )

In [ ]:
delinq_DF.write \
        .format("csv") \
        .option("header", True) \
        .mode("overwrite") \
        .option("path", delinq_csv_dest) \
        .save()

delinq_DF.write \
        .format("parquet") \
        .option("header", True) \
        .mode("overwrite") \
        .option("path", delinq_parq_dest) \
        .save()

In [ ]:
pub_rec.write \
        .format("csv") \
        .option("header", True) \
        .mode("overwrite") \
        .option("path", rec_csv_dest) \
        .save()

pub_rec.write \
        .format("parquet") \
        .option("header", True) \
        .mode("overwrite") \
        .option("path", rec_parq_dest) \
        .save()



In [ ]:
pub_rec_enq_det.write \
        .format("csv") \
        .option("header", True) \
        .mode("overwrite") \
        .option("path", rec_enq_csv_dest) \
        .save()

pub_rec_enq_det.write \
        .format("parquet") \
        .option("header", True) \
        .mode("overwrite") \
        .option("path", rec_enq_parq_dest) \
        .save()



In [ ]:
csv_delinq_count = delinq_DF.count()
addLog("ingested loan_defaulters_delinq file with " + str(csv_delinq_count) + " records in BV")

csv_rec_count = pub_rec.count()
addLog("ingested loan_defaulters_rec file with " + str(csv_rec_count) + " records in BV")

In [ ]:
ldef_stat = "Success"